In [82]:
import gspread
import pandas as pd

import numpy as np
import time
import re
import json
import csv
import math
import pickle

from gspread_dataframe import set_with_dataframe
from selenium import webdriver
from pandas import DataFrame as df

from selenium.webdriver.common.keys import Keys
from selenium.webdriver.common.action_chains import ActionChains
from selenium.webdriver.support.select import Select
from google.oauth2.service_account import Credentials
from webdriver_manager.chrome import ChromeDriverManager


In [83]:
with open('setting.json') as f:
    df = json.load(f)
    SPREADSHEET_KEY = df['KANAGAWA_SPREADSHEET_KEY']
    SECRET_KEY_PATH = df['SECRET_KEY_PATH']

scope = ['https://www.googleapis.com/auth/spreadsheets',
            'https://www.googleapis.com/auth/drive']
credentials = Credentials.from_service_account_file(
    SECRET_KEY_PATH, scopes=scope)
gc = gspread.authorize(credentials)
workbook = gc.open_by_key(SPREADSHEET_KEY)

In [84]:
driver = webdriver.Chrome(ChromeDriverManager().install())




====== WebDriver manager ======
Current google-chrome version is 102.0.5005
Get LATEST chromedriver version for 102.0.5005 google-chrome
Driver [/Users/tatemukaiyuki/.wdm/drivers/chromedriver/mac64/102.0.5005.61/chromedriver] found in cache


In [85]:
def get_list(i):
    URL =  'https://beauty.hotpepper.jp/esthe/svcSA/macJP/salon/PN'+ str(i) +'.html?searchGender=ALL'
    driver.get(URL)
    print( str(i) + "ページ目")

    href_list = []
    title_list = []
    
    hedder_list = driver.find_elements_by_css_selector('.slcHeadContentsInner > h3 > a')
    
    for hedder in hedder_list:
        href_list.append(hedder.get_attribute('href'))
        title_list.append(hedder.text)
        
    return href_list,title_list

In [86]:
def main():
    url = 'https://beauty.hotpepper.jp/esthe/svcSA/macAU/salon/'
    driver.get(url)

    total_items = driver.find_element_by_css_selector('.numberOfResult').text
    area_name = driver.find_element_by_css_selector('#mainContents > div.fgBbrown > h2 > span.fs14.n').text
    
    page_nums = math.ceil(int(total_items) / 20)
    
    print(total_items + '件取得します')
    print(str(page_nums) + 'ページあります')
    
    df_list = []
    for i in range(1, page_nums+1):
        
        href_list, title_list = get_list(i)
        
        df = pd.DataFrame({
            '店名':title_list,
            'URL':href_list
        })
        df_list.append(df)
        
    df_concat = pd.concat(df_list).reset_index(drop = True)
    
    #スプレッドシートに書く
    worksheet = workbook.add_worksheet(title=area_name, rows=1000, cols=20)
    set_with_dataframe(worksheet, df_concat)
    
    driver.quit()
    print('終了')

In [87]:
main()

198件取得します
10ページ
2ページ目
3ページ目
4ページ目
5ページ目
6ページ目
7ページ目
8ページ目
9ページ目
10ページ目
11ページ目
終了


In [45]:
df_list = []
df_list.append(df)
df_concat = pd.concat(df_list).reset_index(drop = True)

In [51]:
area_name = driver.find_element_by_css_selector('#mainContents > div.fgBbrown > h2 > span.fs14.n').text

In [49]:
with open('dataframe.pickle', 'wb') as f:
    pickle.dump(df_concat, f)

In [75]:
workbook.add_worksheet(title=area_name)


TypeError: add_worksheet() missing 2 required positional arguments: 'rows' and 'cols'